In [1]:
#import packages to do EDA
import pandas as pd
import numpy as np

import os
%matplotlib inline
import gc

# http://zetcode.com/python/prettytable/
from prettytable import PrettyTable
import matplotlib.font_manager
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Credit :- https://www.kaggle.com/rinnqd/reduce-memory-usage
def reduce_memory_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
pd.set_option('display.max_columns', None) 

In [4]:
#df_train =  reduce_memory_usage(pd.read_csv('./data/application_train.csv', encoding= 'unicode_escape', nrows=2000))
df_train =  reduce_memory_usage(pd.read_csv('./data/application_train.csv', encoding= 'unicode_escape'))

Memory usage of dataframe is 286.23 MB
Memory usage after optimization is: 92.38 MB
Decreased by 67.7%


In [5]:
#below columns are removed as it is highly correlated with other columns. Refer EDA 
cols=['CNT_FAM_MEMBERS','REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY',
      'LIVE_REGION_NOT_WORK_REGION', 'LIVE_CITY_NOT_WORK_CITY','ELEVATORS_AVG', 'ENTRANCES_AVG','LIVINGAREA_AVG', 
      'APARTMENTS_MODE','YEARS_BEGINEXPLUATATION_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE','FLOORSMAX_MODE',
      'LIVINGAREA_MODE', 'APARTMENTS_MEDI','ELEVATORS_MEDI','ENTRANCES_MEDI','FLOORSMAX_MEDI', 'LIVINGAREA_MEDI', 
      'HOUSETYPE_MODE', 'TOTALAREA_MODE','YEARS_BEGINEXPLUATATION_MEDI','FLOORSMAX_AVG',
      'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_8','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE',
      'BASEMENTAREA_MODE','BASEMENTAREA_MEDI','YEARS_BUILD_MODE','YEARS_BUILD_MEDI','FLOORSMIN_MODE','FLOORSMIN_MEDI',
      'LIVINGAPARTMENTS_MODE','LIVINGAPARTMENTS_MEDI','NONLIVINGAPARTMENTS_MODE','NONLIVINGAPARTMENTS_MEDI',
      'COMMONAREA_MEDI','COMMONAREA_MODE','LANDAREA_MODE','LANDAREA_MEDI','NONLIVINGAREA_MODE','NONLIVINGAREA_MEDI',
      'FLAG_DOCUMENT_2','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_7','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10',
      'FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16',
      'FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21'
     ]
df_train.drop(cols,axis=1,inplace=True)

In [6]:
#fix imputation
def fill_mostfrequent_value(df):
    cat_cols=df.select_dtypes(include=object).columns #categorical values
    for col in cat_cols:
        df[col].fillna(df[col].value_counts(dropna=True).index[0], inplace=True)
    return pd.DataFrame(df)
df_train=fill_mostfrequent_value(df_train)

In [8]:
def fill_median_value(df):
    num_cols=df.select_dtypes(exclude=object).columns
    for col in num_cols:
        df[col].fillna(df[col].median(axis=0,skipna = True),inplace=True)
    return pd.DataFrame(df)
#fill median value for Numerical columns
df_train = fill_median_value(df_train)

In [10]:
df_train['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

In [11]:
#before function convert negative values to postive and convert them into years. 
def convert_negative_days_year(days_negative):
    #convert negative values to positive, and divide by 365 days to convert years
    #finally rounded to 1 decimal.
    return np.round(abs(days_negative)/365,1)

#Train
df_train['Age'] = convert_negative_days_year(df_train['DAYS_BIRTH'])
df_train['Employment'] = convert_negative_days_year(df_train['DAYS_EMPLOYED'])
df_train['Phone_change_Years'] = convert_negative_days_year(df_train['DAYS_LAST_PHONE_CHANGE'])
df_train['Registration_Years'] = convert_negative_days_year(df_train['DAYS_REGISTRATION'])
df_train['Id_Publish_Years'] = convert_negative_days_year(df_train['DAYS_ID_PUBLISH'])

In [12]:
#drop these columns from train dataset
df_train.drop(['DAYS_BIRTH','DAYS_EMPLOYED','DAYS_LAST_PHONE_CHANGE','DAYS_REGISTRATION','DAYS_ID_PUBLISH'],axis=1,inplace=True)

In [13]:
#Try to add some more features domain based
# Credit :- https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction#Feature-Engineering
epsilon=0.001 # to avoid div/0 error 
def Feature_Engineering(df):
    df1 = pd.DataFrame()
    df1['SK_ID_CURR']=df['SK_ID_CURR']
    df1['pc_Credit_Income'] = np.round((df['AMT_CREDIT'] / (df['AMT_INCOME_TOTAL']+epsilon)),4)
    df1['pc_Annuity_Income'] = np.round((df['AMT_ANNUITY'] / (df['AMT_INCOME_TOTAL']+epsilon)),4)
    df1['pc_Credit_Annuity'] = np.round((df['AMT_CREDIT'] / (df['AMT_ANNUITY']+epsilon)),4)
    df1['Credit_Goods_Diff'] = df['AMT_CREDIT'] - df['AMT_GOODS_PRICE']
    df1['pc_Loan_Value']=np.round((df['AMT_CREDIT'] / (df['AMT_GOODS_PRICE']+epsilon)),4)
    df1['CREDIT_TERM'] = np.round((df['AMT_ANNUITY'] / (df['AMT_CREDIT']+epsilon)),4)
    df1['pc_Employment_Age'] = np.round((df['Employment'] / (df['Age']+epsilon)),4)
    return df1

df2=Feature_Engineering(df_train)
df_train = df_train.merge(df2, on = 'SK_ID_CURR', how = 'left')
print('Train data after Feature Engineering ~>{}'.format(df_train.shape))

Train data after Feature Engineering ~>(307511, 69)


In [14]:
# Merge new features into main dataframe
gc.enable()
del df2
gc.collect()

8

In [15]:
df_train['EXT_SOURCE_1'] = df_train['EXT_SOURCE_1'].round(decimals=4)
df_train['EXT_SOURCE_2']=df_train['EXT_SOURCE_2'].round(decimals=4)
df_train['EXT_SOURCE_3']=df_train['EXT_SOURCE_3'].round(decimals=4)
#RuntimeWarning: overflow encountered in multiply
#while transforming the below features to polinomial, runtime overflow occurs
#to avoid, convert all of them to float64
df_train['EXT_SOURCE_1'] = df_train['EXT_SOURCE_1'].astype(np.float64)
df_train['EXT_SOURCE_2'] = df_train['EXT_SOURCE_2'].astype(np.float64)
df_train['EXT_SOURCE_3'] = df_train['EXT_SOURCE_3'].astype(np.float64)
df_train['Age'] = df_train['Age'].astype(np.float64)
df_train['Employment'] = df_train['Employment'].astype(np.float64)

In [16]:
categorical_columns=df_train.select_dtypes(include=object).columns.to_list()
numerical_columns=df_train.select_dtypes(exclude=object).columns.to_list()
print('# of Numerical columns ~~>{} and # of Categorical columns ~~> {} '.format(len(numerical_columns),len(categorical_columns)))

# of Numerical columns ~~>54 and # of Categorical columns ~~> 15 


In [17]:
# Try to add some polinomial features

# Make a new dataframe for polynomial features
#credit: https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction#Feature-Engineering
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

# Generate a new feature matrix consisting of all polynomial combinations of the features with degree less than 
# or equal to the specified degree. For example, if an input sample is two dimensional and of the form [a, b], 
# the degree-2 polynomial features are [1, a, b, a^2, ab, b^2].

# imputer for handling missing values
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='median')

poly_features = df_train[['SK_ID_CURR','EXT_SOURCE_1','EXT_SOURCE_2', 'EXT_SOURCE_3', 'Age', 'Employment','TARGET']]

poly_target = poly_features['TARGET']
poly_features = poly_features.drop(columns = ['TARGET'])

# Need to impute missing values
poly_features = imputer.fit_transform(poly_features[['EXT_SOURCE_1','EXT_SOURCE_2', 'EXT_SOURCE_3', 'Age','Employment']])

# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 2)

# Train the polynomial features
poly_features= poly_transformer.fit_transform(poly_features)

print('Polynomial Features shape: ', poly_features.shape)

Polynomial Features shape:  (307511, 21)


In [18]:
poly_features = pd.DataFrame(poly_features, 
                            columns = poly_transformer.get_feature_names(
                                            input_features = ['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3', 'Age','Employment']))
poly_features['SK_ID_CURR'] = df_train['SK_ID_CURR']

# Merge polynomial features into training dataframe
df_train = df_train.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

In [19]:
print(df_train.shape)

(307511, 90)


#### Merge Bureau, Bureau Balance to Train

In [20]:
# we will put together all above bureau functions, into one function. So that, we can use it for 
# other dataframes
# credit:https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering
def numeric_aggregate(df, group_var, df_name):
    """Aggregates the numeric values in a dataframe. This can
    be used to create features for each instance of the grouping variable.
    
    Parameters
    --------
        df (dataframe): 
            the dataframe to calculate the statistics on
        group_var (string): 
            the variable by which to group df
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated for 
            all numeric columns. Each instance of the grouping variable will have 
            the statistics (mean, min, max, sum; currently supported) calculated. 
            The columns are also renamed to keep track of features created.
    
    """
    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

    # Need to create new column names
    columns = [group_var]

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    return agg

In [21]:
# calculate the counts and normalized counts of each category for all categorical variables in the dataframe.
# very similar to numeric_aggregate
## credit:https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering

def feature_engineering_categorical_variables(df, group_var, df_name):
    """Computes counts and normalized counts for each observation
    of `group_var` of each unique category in every categorical variable
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    group_var : string
        The variable by which to group the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with counts and normalized counts of each unique category in every categorical variable
        with one row for every unique value of the `group_var`.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

In [22]:
df_bureau = reduce_memory_usage(pd.read_csv('./data/bureau.csv', encoding= 'unicode_escape'))
print('Number of data points : ', df_bureau.shape[0])
print('Number of features : ', df_bureau.shape[1])
df_bureau.head(1)

Memory usage of dataframe is 222.62 MB
Memory usage after optimization is: 112.95 MB
Decreased by 49.3%
Number of data points :  1716428
Number of features :  17


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN


In [23]:
#Check bureau and train have similar columns. if so, rename bureau columns, except ID columns
print('Similar columns~~>{}'.format(df_bureau.columns.intersection(df_train.columns).tolist()))

Similar columns~~>['SK_ID_CURR', 'AMT_ANNUITY']


In [24]:
df_bureau.rename(columns={"AMT_ANNUITY": "BUREAU_AMT_ANNUITY"}, inplace = True)

In [25]:
num_columns=df_bureau.select_dtypes(exclude=object).columns.to_list()
print('# of Numerical columns in Bureau~~> {}'.format(len(num_columns)))

cat_columns=df_bureau.select_dtypes(include=object).columns.to_list()
print('# of categorical columns in Bureau~~> {}'.format(len(cat_columns)))
cat_columns.append('SK_ID_CURR')

# of Numerical columns in Bureau~~> 14
# of categorical columns in Bureau~~> 3


In [26]:
df_days_cols = df_bureau.loc[:, df_bureau.columns.str.contains('DAY')]
df_days_cols.columns

Index(['DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE'],
      dtype='object')

In [27]:
for col in df_days_cols:
    print('column {} min value {} and max valaue {}'.format(col, df_days_cols[col].min(),df_days_cols[col].max()))

column DAYS_CREDIT min value -2922 and max valaue 0
column CREDIT_DAY_OVERDUE min value 0 and max valaue 2792
column DAYS_CREDIT_ENDDATE min value -42048.0 and max valaue 31200.0
column DAYS_ENDDATE_FACT min value -42016.0 and max valaue 0.0
column DAYS_CREDIT_UPDATE min value -41947 and max valaue 372


In [28]:
df_bureau['DAYS_CREDIT']=-df_bureau['DAYS_CREDIT']
df_bureau['DAYS_ENDDATE_FACT']=-df_bureau['DAYS_ENDDATE_FACT']

In [29]:
# fill missing values
df_bureau=fill_mostfrequent_value(df_bureau)
df_bureau = fill_median_value(df_bureau)

In [30]:
print(df_bureau[df_bureau.isin([np.nan, np.inf, -np.inf]).any(1)].shape[0])

0


In [31]:
# Group by the applicant id, calculate aggregation statistics
df_bureau_agg = numeric_aggregate(df_bureau.drop(columns = ['SK_ID_BUREAU']),group_var = 'SK_ID_CURR', df_name = 'bureau')
df_bureau_agg.head(1)

,SK_ID_CURR,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_sum,bureau_CREDIT_DAY_OVERDUE_count,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,bureau_CREDIT_DAY_OVERDUE_min,bureau_CREDIT_DAY_OVERDUE_sum,bureau_DAYS_CREDIT_ENDDATE_count,bureau_DAYS_CREDIT_ENDDATE_mean,bureau_DAYS_CREDIT_ENDDATE_max,bureau_DAYS_CREDIT_ENDDATE_min,bureau_DAYS_CREDIT_ENDDATE_sum,bureau_DAYS_ENDDATE_FACT_count,bureau_DAYS_ENDDATE_FACT_mean,bureau_DAYS_ENDDATE_FACT_max,bureau_DAYS_ENDDATE_FACT_min,bureau_DAYS_ENDDATE_FACT_sum,bureau_AMT_CREDIT_MAX_OVERDUE_count,bureau_AMT_CREDIT_MAX_OVERDUE_mean,bureau_AMT_CREDIT_MAX_OVERDUE_max,bureau_AMT_CREDIT_MAX_OVERDUE_min,bureau_AMT_CREDIT_MAX_OVERDUE_sum,bureau_CNT_CREDIT_PROLONG_count,bureau_CNT_CREDIT_PROLONG_mean,bureau_CNT_CREDIT_PROLONG_max,bureau_CNT_CREDIT_PROLONG_min,bureau_CNT_CREDIT_PROLONG_sum,bureau_AMT_CREDIT_SUM_count,bureau_AMT_CREDIT_SUM_mean,bureau_AMT_CREDIT_SUM_max,bureau_AMT_CREDIT_SUM_min,bureau_AMT_CREDIT_SUM_sum,bureau_AMT_CREDIT_SUM_DEBT_count,bureau_AMT_CREDIT_SUM_DEBT_mean,bureau_AMT_CREDIT_SUM_DEBT_max,bureau_AMT_CREDIT_SUM_DEBT_min,bureau_AMT_CREDIT_SUM_DEBT_sum,bureau_AMT_CREDIT_SUM_LIMIT_count,bureau_AMT_CREDIT_SUM_LIMIT_mean,bureau_AMT_CREDIT_SUM_LIMIT_max,bureau_AMT_CREDIT_SUM_LIMIT_min,bureau_AMT_CREDIT_SUM_LIMIT_sum,bureau_AMT_CREDIT_SUM_OVERDUE_count,bureau_AMT_CREDIT_SUM_OVERDUE_mean,bureau_AMT_CREDIT_SUM_OVERDUE_max,bureau_AMT_CREDIT_SUM_OVERDUE_min,bureau_AMT_CREDIT_SUM_OVERDUE_sum,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_sum,bureau_BUREAU_AMT_ANNUITY_count,bureau_BUREAU_AMT_ANNUITY_mean,bureau_BUREAU_AMT_ANNUITY_max,bureau_BUREAU_AMT_ANNUITY_min,bureau_BUREAU_AMT_ANNUITY_sum
0,100001,7,735.0,1572,49,5145.0,7,0.0,0,0,0,7,82.4375,1778.0,-1329.0,577.0,7,856.0,1328.0,544.0,5992.0,7,0.0,0.0,0.0,0.0,7,0.0,0,0,0,7,207623.578125,378000.0,85500.0,1453365.0,7,85240.929688,373239.0,0.0,596686.5,7,0.0,0.0,0.0,0.0,7,0.0,0.0,0.0,0.0,7,-93.142857,-6,-155,-652,7,3545.357178,10822.5,0.0,24817.5


In [32]:
df_bureau_categorical = feature_engineering_categorical_variables(df=df_bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')
df_bureau_categorical.head(1)

,bureau_CREDIT_ACTIVE_Active_count,bureau_CREDIT_ACTIVE_Active_count_norm,bureau_CREDIT_ACTIVE_Bad debt_count,bureau_CREDIT_ACTIVE_Bad debt_count_norm,bureau_CREDIT_ACTIVE_Closed_count,bureau_CREDIT_ACTIVE_Closed_count_norm,bureau_CREDIT_ACTIVE_Sold_count,bureau_CREDIT_ACTIVE_Sold_count_norm,bureau_CREDIT_CURRENCY_currency 1_count,bureau_CREDIT_CURRENCY_currency 1_count_norm,bureau_CREDIT_CURRENCY_currency 2_count,bureau_CREDIT_CURRENCY_currency 2_count_norm,bureau_CREDIT_CURRENCY_currency 3_count,bureau_CREDIT_CURRENCY_currency 3_count_norm,bureau_CREDIT_CURRENCY_currency 4_count,bureau_CREDIT_CURRENCY_currency 4_count_norm,bureau_CREDIT_TYPE_Another type of loan_count,bureau_CREDIT_TYPE_Another type of loan_count_norm,bureau_CREDIT_TYPE_Car loan_count,bureau_CREDIT_TYPE_Car loan_count_norm,bureau_CREDIT_TYPE_Cash loan (non-earmarked)_count,bureau_CREDIT_TYPE_Cash loan (non-earmarked)_count_norm,bureau_CREDIT_TYPE_Consumer credit_count,bureau_CREDIT_TYPE_Consumer credit_count_norm,bureau_CREDIT_TYPE_Credit card_count,bureau_CREDIT_TYPE_Credit card_count_norm,bureau_CREDIT_TYPE_Interbank credit_count,bureau_CREDIT_TYPE_Interbank credit_count_norm,bureau_CREDIT_TYPE_Loan for business development_count,bureau_CREDIT_TYPE_Loan for business development_count_norm,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_count,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_count_norm,bureau_CREDIT_TYPE_Loan for the purchase of equipment_count,bureau_CREDIT_TYPE_Loan for the purchase of equipment_count_norm,bureau_CREDIT_TYPE_Loan for working capital replenishment_count,bureau_CREDIT_TYPE_Loan for working capital replenishment_count_norm,bureau_CREDIT_TYPE_Microloan_count,bureau_CREDIT_TYPE_Microloan_count_norm,bureau_CREDIT_TYPE_Mobile operator loan_count,bureau_CREDIT_TYPE_Mobile operator loan_count_norm,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_count_norm
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,3,0.428571,0,0.0,4,0.571429,0,0.0,7,1.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,7,1.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [33]:
df_bureau_agg.replace([np.inf, -np.inf,np.NaN], 0, inplace=True)

In [34]:
print(df_bureau_agg[df_bureau_agg.isin([np.nan, np.inf, -np.inf]).any(1)].shape[0])
#before imputing, replace infinite values with np.nan values. Inputer will update with median values
print(df_bureau_categorical[df_bureau_categorical.isin([np.nan, np.inf, -np.inf]).any(1)].shape[0])

0
0


In [35]:
#merge df_bureau categorical columns with train since it is multiple take the 1 record per I
df_bureau_categorical_group =df_bureau[cat_columns].groupby('SK_ID_CURR').head(1).reset_index(drop=True)
df_train=df_train.merge(df_bureau_categorical_group,on='SK_ID_CURR', how='left' )

In [36]:
# Merge bureau numerical aggregation with the training data
df_train = df_train.merge(df_bureau_agg, on = 'SK_ID_CURR', how = 'left')
# Merge bureau categorical numerical data with the training data
df_train = df_train.merge(df_bureau_categorical, on = 'SK_ID_CURR', how = 'left')
#df_train = reduce_memory_usage(df_train)
print('Number of data points : ', df_train.shape[0])
print('Number of features : ', df_train.shape[1])

Number of data points :  307511
Number of features :  199


In [37]:
#check if any records with nan or inf value before 
#zero means no inf, nan values
df_train.replace([np.inf, -np.inf,np.NaN], 0, inplace=True)
#df_train[df_train.isin([np.nan, np.inf, -np.inf]).any(1)].shape[0]

#### Bureau Balance dataset

In [38]:
df_bureau_balance =  reduce_memory_usage(pd.read_csv('./data/bureau_balance.csv', encoding= 'unicode_escape'))
print('Number of data points : ', df_bureau_balance.shape[0])
print('Number of features : ', df_bureau_balance.shape[1])
df_bureau_balance.head(1)

Memory usage of dataframe is 624.85 MB
Memory usage after optimization is: 338.46 MB
Decreased by 45.8%
Number of data points :  27299925
Number of features :  3


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C


In [39]:
#since the bureau balance STATUS has one letter, and have multiple status per SK_ID_CURR, 
#it is hard to merge at the SK_ID_CURR. it is better to ignore / don't merge with train dataset
#will add Status based categorical numerical addition

In [40]:
num_columns=df_bureau_balance.select_dtypes(exclude=object).columns.to_list()
print('# of Numerical columns in Bureau Balance~~> {}'.format(len(num_columns)))

cat_columns=df_bureau_balance.select_dtypes(include=object).columns.to_list()
print('# of categorical columns in Bureau Balance~~> {}'.format(len(cat_columns)))
cat_columns.append('SK_ID_CURR')

# of Numerical columns in Bureau Balance~~> 2
# of categorical columns in Bureau Balance~~> 1


In [41]:
df_days_cols = df_bureau_balance.loc[:, df_bureau_balance.columns.str.contains('BALANCE')]
df_days_cols.columns
for col in df_days_cols:
    print('column {} min value {} and max valaue {}'.format(col, df_days_cols[col].min(),df_days_cols[col].max()))

column MONTHS_BALANCE min value -96 and max valaue 0


In [42]:
#since overdue doesn't have negative values, convert them into positive
df_bureau_balance['MONTHS_BALANCE'] = -df_bureau_balance['MONTHS_BALANCE']

In [43]:
#Check bureau and train have similar columns. if so, rename bureau columns, except ID columns
print('Similar columns~~>{}'.format(df_bureau_balance.columns.intersection(df_train.columns).tolist()))
#Check bureau and bureau have similar columns. if so, rename bureau columns, except ID columns
print('Similar columns~~>{}'.format(df_bureau_balance.columns.intersection(df_bureau.columns).tolist()))

Similar columns~~>[]
Similar columns~~>['SK_ID_BUREAU']


In [44]:
# fill missing values
df_bureau_balance=fill_mostfrequent_value(df_bureau_balance)
df_bureau_balance = fill_median_value(df_bureau_balance)

In [45]:
df_bureau_balance_agg = numeric_aggregate(df=df_bureau_balance,group_var = 'SK_ID_BUREAU', df_name = 'bureau_bal')
#Add Categorical field numerical counts
df_bureau_balance_categorical = feature_engineering_categorical_variables(df=df_bureau_balance, 
                                                                      group_var = 'SK_ID_BUREAU', df_name = 'bureau_bal')
# Dataframe grouped by the loan
bureau_by_loan = df_bureau_balance_agg.merge(df_bureau_balance_categorical, right_index = True, 
                                             left_on = 'SK_ID_BUREAU', how = 'left')
# Merge to include the SK_ID_CURR
bureau_by_loan = df_bureau[['SK_ID_BUREAU', 'SK_ID_CURR']]\
                            .merge(bureau_by_loan, on = 'SK_ID_BUREAU', how = 'left').reset_index(drop=True)
#drop SK_ID_BUREAU column and aggreate at client level
bureau_by_loan.drop(columns = ['SK_ID_BUREAU'],axis=1,inplace=True)
# Aggregate the stats for each client
bureau_balance_by_client = numeric_aggregate(df=bureau_by_loan,group_var = 'SK_ID_CURR', df_name = 'client')

# Merge with the training data
df_train = df_train.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left').reset_index(drop=True)

In [46]:
#check if any records with nan or inf value before 
#zero means no inf, nan values
df_train.replace([np.inf, -np.inf,np.NaN], 0, inplace=True)
#df_train[df_train.isin([np.nan, np.inf, -np.inf]).any(1)].shape[0]

#### Previous Loan count from Bureau dataset

In [47]:
prev_loan_cnt = df_bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().\
                                                rename(columns = {'SK_ID_BUREAU': 'prev_loan_cnt'})
#merge previous loan counts to train dataset
df_train = df_train.merge(prev_loan_cnt, on = 'SK_ID_CURR', how = 'left')

In [48]:
#check if any records with nan or inf value before 
#zero means no inf, nan values
df_train.replace([np.inf, -np.inf,np.NaN], 0, inplace=True)
#df_train[df_train.isin([np.nan, np.inf, -np.inf]).any(1)].shape[0]

In [49]:
gc.enable()
del df_bureau,df_bureau_agg,df_bureau_categorical,df_bureau_balance,df_bureau_balance_agg,\
    df_bureau_balance_categorical,prev_loan_cnt,df_bureau_categorical_group
gc.collect()

11

####  Previous Application Dataset

In [50]:
df_previous_application =reduce_memory_usage(pd.read_csv('./data/previous_application.csv', encoding= 'unicode_escape'))
print('Number of data points : ', df_previous_application.shape[0])
print('Number of features : ', df_previous_application.shape[1])
df_previous_application.head(1)

Memory usage of dataframe is 471.48 MB
Memory usage after optimization is: 309.01 MB
Decreased by 34.5%
Number of data points :  1670214
Number of features :  37


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430054,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182861,0.867188,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0


In [51]:
#Check previous application and current train dataset have similar columns. if so, rename pre_app columns, except ID columns
print('Similar columns~~>{}'.format(df_previous_application.columns.intersection(df_train.columns).tolist()))

Similar columns~~>['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'NAME_TYPE_SUITE']


In [52]:
df_previous_application.rename(columns={"NAME_CONTRACT_TYPE": "pre_app_NAME_CONTRACT_TYPE"}, inplace = True)
df_previous_application.rename(columns={"AMT_ANNUITY": "pre_app_AMT_ANNUITY"}, inplace = True)
df_previous_application.rename(columns={"AMT_CREDIT": "pre_app_AMT_CREDIT"}, inplace = True)
df_previous_application.rename(columns={"AMT_GOODS_PRICE": "pre_app_AMT_GOODS_PRICEE"}, inplace = True)
df_previous_application.rename(columns={"WEEKDAY_APPR_PROCESS_START": "pre_app_WEEKDAY_APPR_PROCESS_START"}, inplace = True)
df_previous_application.rename(columns={"HOUR_APPR_PROCESS_START": "pre_app_HOUR_APPR_PROCESS_START"}, inplace = True)
df_previous_application.rename(columns={"NAME_TYPE_SUITE": "pre_app_NAME_TYPE_SUITE"}, inplace = True)

In [53]:
df_days_cols = df_previous_application.loc[:, df_previous_application.columns.str.contains('AMT')]
df_days_cols.columns
for col in df_days_cols:
    print('column {} min value {} and max valaue {}'.format(col, df_days_cols[col].min(),df_days_cols[col].max()))

column pre_app_AMT_ANNUITY min value 0.0 and max valaue 418058.15625
column AMT_APPLICATION min value 0.0 and max valaue 6905160.0
column pre_app_AMT_CREDIT min value 0.0 and max valaue 6905160.0
column AMT_DOWN_PAYMENT min value -0.8999999761581421 and max valaue 3060045.0
column pre_app_AMT_GOODS_PRICEE min value 0.0 and max valaue 6905160.0


In [54]:
#convert negative values to zero as there is not concept of negative down payment
df_previous_application.loc[df_previous_application['AMT_DOWN_PAYMENT'] <0,'AMT_DOWN_PAYMENT']=0

In [55]:
df_days_cols = df_previous_application.loc[:, df_previous_application.columns.str.contains('DAY')]
for col in df_days_cols:
    print('column {} min value {} and max valaue {}'.format(col, df_days_cols[col].min(),df_days_cols[col].max()))

column pre_app_WEEKDAY_APPR_PROCESS_START min value FRIDAY and max valaue WEDNESDAY
column NFLAG_LAST_APPL_IN_DAY min value 0 and max valaue 1
column DAYS_DECISION min value -2922 and max valaue -1
column DAYS_FIRST_DRAWING min value -2922.0 and max valaue 365243.0
column DAYS_FIRST_DUE min value -2892.0 and max valaue 365243.0
column DAYS_LAST_DUE_1ST_VERSION min value -2801.0 and max valaue 365243.0
column DAYS_LAST_DUE min value -2889.0 and max valaue 365243.0
column DAYS_TERMINATION min value -2874.0 and max valaue 365243.0


In [56]:
#DAYS DECISION will be +ve days
df_previous_application['DAYS_DECISION']=-df_previous_application['DAYS_DECISION']

In [57]:
num_columns=df_previous_application.select_dtypes(exclude=object).columns.to_list()
print('# of Numerical columns in previous application~~> {}'.format(len(num_columns)))

cat_columns=df_previous_application.select_dtypes(include=object).columns.to_list()
print('# of categorical columns in previous application~~> {}'.format(len(cat_columns)))
cat_columns.append('SK_ID_CURR')

# of Numerical columns in previous application~~> 21
# of categorical columns in previous application~~> 16


In [58]:
# fill missing values
df_previous_application=fill_mostfrequent_value(df_previous_application)
df_previous_application = fill_median_value(df_previous_application)

In [59]:
#merge previous application categorical columns with train since it is multiple take the 1 record per ID
df_Pre_App_cat_group =df_previous_application[cat_columns].groupby('SK_ID_CURR').head(1).reset_index(drop=True)
df_train=df_train.merge(df_Pre_App_cat_group,on='SK_ID_CURR', how='left' )

In [60]:
#newly added categorical columns will have nan values. Hence fill with most frequent values 
df_train=fill_mostfrequent_value(df_train)

In [61]:
# Group by the applicant id, calculate aggregation statistics
df_prev_app_numerical_agg = numeric_aggregate(df_previous_application.drop(columns = ['SK_ID_PREV']),
                                              group_var = 'SK_ID_CURR', df_name = 'pre_app').reset_index(drop=True)
#merge with df_train
df_train=df_train.merge(df_prev_app_numerical_agg,on='SK_ID_CURR', how='left')

In [62]:
df_prev_app_cat_agg = feature_engineering_categorical_variables(df=df_previous_application, 
                                              group_var = 'SK_ID_CURR', df_name = 'pre_app').reset_index()
#merge with df_train
df_train=df_train.merge(df_prev_app_cat_agg,on='SK_ID_CURR', how='left' )

In [63]:
#check if any records with nan or inf value before 
#zero means no inf, nan values
df_train.replace([np.inf, -np.inf,np.NaN], 0, inplace=True)
#df_train[df_train.isin([np.nan, np.inf, -np.inf]).any(1)].shape[0]

In [64]:
gc.enable()
del df_prev_app_cat_agg,df_prev_app_numerical_agg,df_previous_application,df_Pre_App_cat_group
gc.collect()

11

#### Monthly Cash

In [65]:
df_cash_balance = reduce_memory_usage(pd.read_csv('./data/POS_CASH_balance.csv', encoding= 'unicode_escape'))
print('Number of data points : ', df_cash_balance.shape[0])
print('Number of features : ', df_cash_balance.shape[1])
df_cash_balance.head(1)

Memory usage of dataframe is 610.43 MB
Memory usage after optimization is: 238.45 MB
Decreased by 60.9%
Number of data points :  10001358
Number of features :  8


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0


In [66]:
#Check bureau and train have similar columns. if so, rename bureau columns, except ID columns
print('Similar columns~~>{}'.format(df_cash_balance.columns.intersection(df_train.columns).tolist()))

Similar columns~~>['SK_ID_CURR', 'NAME_CONTRACT_STATUS']


In [67]:
df_cash_balance.rename(columns={"NAME_CONTRACT_STATUS": "Cash_Bal_NAME_CONTRACT_STATUS"}, inplace = True)
df_cash_balance.rename(columns={"MONTHS_BALANCE": "Cash_Bal_MONTHS_BALANCE"}, inplace = True)

In [68]:
num_columns=df_cash_balance.select_dtypes(exclude=object).columns.to_list()
print('# of Numerical columns in previous application~~> {}'.format(len(num_columns)))

cat_columns=df_cash_balance.select_dtypes(include=object).columns.to_list()
print('# of categorical columns in previous application~~> {}'.format(len(cat_columns)))
cat_columns.append('SK_ID_CURR')

# of Numerical columns in previous application~~> 7
# of categorical columns in previous application~~> 1


In [69]:
df_cols = df_cash_balance.loc[:, df_cash_balance.columns.str.contains('BALANCE')]
for col in df_cols:
    print('column {} min value {} and max valaue {}'.format(col, df_cols[col].min(),df_cols[col].max()))

column Cash_Bal_MONTHS_BALANCE min value -96 and max valaue -1


In [70]:
df_cash_balance['Cash_Bal_MONTHS_BALANCE']=-df_cash_balance['Cash_Bal_MONTHS_BALANCE']

In [71]:
df_cols = df_cash_balance.loc[:, df_cash_balance.columns.str.contains('INSTAL')]
for col in df_cols:
    print('column {} min value {} and max valaue {}'.format(col, df_cols[col].min(),df_cols[col].max()))

column CNT_INSTALMENT min value 1.0 and max valaue 92.0
column CNT_INSTALMENT_FUTURE min value 0.0 and max valaue 85.0


In [72]:
# fill missing values
df_cash_balance=fill_mostfrequent_value(df_cash_balance)
df_cash_balance = fill_median_value(df_cash_balance)

In [73]:
#merge previous application categorical columns with train since it is multiple take the 1 record per ID
df_Cash_Bal_cat_group =df_cash_balance[cat_columns].groupby('SK_ID_CURR').head(1).reset_index(drop=True)
df_train=df_train.merge(df_Cash_Bal_cat_group,on='SK_ID_CURR', how='left')

In [74]:
df_train=fill_mostfrequent_value(df_train)

In [75]:
# Group by the applicant id, calculate aggregation statistics
df_cash_bal_numerical_agg = numeric_aggregate(df_cash_balance.drop(columns = ['SK_ID_PREV']),
                                              group_var = 'SK_ID_CURR', df_name = 'cash_bal').reset_index(drop=True)
                                    
df_train=df_train.merge(df_cash_bal_numerical_agg,on='SK_ID_CURR', how='left' )
df_cash_bal_cat_agg = feature_engineering_categorical_variables(df=df_cash_balance, 
                                                        group_var = 'SK_ID_CURR', df_name = 'cash_bal').reset_index()
df_train=df_train.merge(df_cash_bal_cat_agg,on='SK_ID_CURR', how='left' )

In [76]:
#check if any records with nan or inf value before 
#zero means no inf, nan values
df_train.replace([np.inf, -np.inf,np.NaN], 0, inplace=True)
#df_train[df_train.isin([np.nan, np.inf, -np.inf]).any(1)].shape[0]

In [77]:
# Remove variables to free memory
gc.enable()
del df_cash_balance, df_cash_bal_numerical_agg, df_cash_bal_cat_agg,df_Cash_Bal_cat_group
gc.collect()

11

#### Monthly Credit Data

In [78]:
df_CCard_balance = reduce_memory_usage(pd.read_csv('./data/credit_card_balance.csv', encoding= 'unicode_escape'))
print('Number of data points : ', df_CCard_balance.shape[0])
print('Number of features : ', df_CCard_balance.shape[1])

Memory usage of dataframe is 673.88 MB
Memory usage after optimization is: 289.33 MB
Decreased by 57.1%
Number of data points :  3840312
Number of features :  23


In [79]:
df_CCard_balance.head(1)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,1800.0,1800.0,0.0,0.0,0.0,0.0,1,0.0,1.0,35.0,Active,0,0


In [80]:
df_cols = df_CCard_balance.loc[:, df_CCard_balance.columns.str.contains('BALANCE')]
for col in df_cols:
    print('column {} min value {} and max valaue {}'.format(col, df_cols[col].min(),df_cols[col].max()))

column MONTHS_BALANCE min value -96 and max valaue -1
column AMT_BALANCE min value -420250.1875 and max valaue 1505902.125


In [81]:
df_CCard_balance['MONTHS_BALANCE']=-df_CCard_balance['MONTHS_BALANCE']
#convert negative values to zero as there is not concept of negative balance amount
df_CCard_balance.loc[df_CCard_balance['AMT_BALANCE'] <0,'AMT_BALANCE']=0


In [82]:
df_cols = df_CCard_balance.loc[:, df_CCard_balance.columns.str.contains('AMT')]
for col in df_cols:
    print('column {} min value {} and max valaue {}'.format(col, df_cols[col].min(),df_cols[col].max()))

column AMT_BALANCE min value 0.0 and max valaue 1505902.125
column AMT_CREDIT_LIMIT_ACTUAL min value 0 and max valaue 1350000
column AMT_DRAWINGS_ATM_CURRENT min value -6827.31005859375 and max valaue 2115000.0
column AMT_DRAWINGS_CURRENT min value -6211.6201171875 and max valaue 2287098.25
column AMT_DRAWINGS_OTHER_CURRENT min value 0.0 and max valaue 1529847.0
column AMT_DRAWINGS_POS_CURRENT min value 0.0 and max valaue 2239274.25
column AMT_INST_MIN_REGULARITY min value 0.0 and max valaue 202882.0
column AMT_PAYMENT_CURRENT min value 0.0 and max valaue 4289207.5
column AMT_PAYMENT_TOTAL_CURRENT min value 0.0 and max valaue 4278315.5
column AMT_RECEIVABLE_PRINCIPAL min value -423305.8125 and max valaue 1472316.75
column AMT_RECIVABLE min value -420250.1875 and max valaue 1493338.125
column AMT_TOTAL_RECEIVABLE min value -420250.1875 and max valaue 1493338.125


In [83]:
#-VE TO BE CHANGED TO 0. Drawings or receivables can't be -ve. 
df_CCard_balance.loc[df_CCard_balance['AMT_DRAWINGS_ATM_CURRENT'] <0,'AMT_DRAWINGS_ATM_CURRENT']=0
df_CCard_balance.loc[df_CCard_balance['AMT_DRAWINGS_CURRENT'] <0,'AMT_DRAWINGS_CURRENT']=0
df_CCard_balance.loc[df_CCard_balance['AMT_RECEIVABLE_PRINCIPAL'] <0,'AMT_RECEIVABLE_PRINCIPAL']=0
df_CCard_balance.loc[df_CCard_balance['AMT_RECIVABLE'] <0,'AMT_RECIVABLE']=0
df_CCard_balance.loc[df_CCard_balance['AMT_TOTAL_RECEIVABLE'] <0,'AMT_TOTAL_RECEIVABLE ']=0 

In [84]:
#Check bureau and train have similar columns. if so, rename bureau columns, except ID columns
print('Similar columns~~>{}'.format(df_CCard_balance.columns.intersection(df_train.columns).tolist()))

Similar columns~~>['SK_ID_CURR', 'NAME_CONTRACT_STATUS']


In [85]:
#since NAME_CONTRACT_STATUS is repeated. rename this column
df_CCard_balance.rename(columns = {'NAME_CONTRACT_STATUS':'CCard_NAME_CONTRACT_STATUS'}, inplace = True)
df_CCard_balance.rename(columns = {'MONTHS_BALANCE':'CCard_MONTHS_BALANCE'}, inplace = True)

In [86]:
num_columns=df_CCard_balance.select_dtypes(exclude=object).columns.to_list()
print('# of Numerical columns in previous application~~> {}'.format(len(num_columns)))

cat_columns=df_CCard_balance.select_dtypes(include=object).columns.to_list()
print('# of categorical columns in previous application~~> {}'.format(len(cat_columns)))
cat_columns.append('SK_ID_CURR')

# of Numerical columns in previous application~~> 23
# of categorical columns in previous application~~> 1


In [87]:
df_CCard_balance[cat_columns].head(1)

,CCard_NAME_CONTRACT_STATUS,SK_ID_CURR
0,Active,378907


In [88]:
# fill missing values
df_CCard_balance=fill_mostfrequent_value(df_CCard_balance)
df_CCard_balance = fill_median_value(df_CCard_balance)

In [89]:
#merge previous application categorical columns with train since it is multiple take the 1 record per ID
df_CCard_Bal_cat_group =df_CCard_balance[cat_columns].groupby('SK_ID_CURR').head(1).reset_index(drop=True)
df_train=df_train.merge(df_CCard_Bal_cat_group,on='SK_ID_CURR', how='left')

In [90]:
# fill missing values
df_train=fill_mostfrequent_value(df_train)

In [91]:
# Group by the applicant id, calculate aggregation statistics
df_CCard_bal_numerical_agg = numeric_aggregate(df_CCard_balance.drop(columns = ['SK_ID_PREV']),
                                              group_var = 'SK_ID_CURR', df_name = 'CC_bal').reset_index(drop=True)
df_train=df_train.merge(df_CCard_bal_numerical_agg,on='SK_ID_CURR', how='left' )

df_CCard_bal_cat_agg = feature_engineering_categorical_variables(df=df_CCard_balance, 
                                                        group_var = 'SK_ID_CURR', df_name = 'CC_bal').reset_index()
df_train=df_train.merge(df_CCard_bal_cat_agg,on='SK_ID_CURR', how='left' )

In [92]:
#check if any records with nan or inf value before 
#zero means no inf, nan values
df_train.replace([np.inf, -np.inf,np.NaN], 0, inplace=True)
#df_train[df_train.isin([np.nan, np.inf, -np.inf]).any(1)].shape[0]

In [93]:
gc.enable()
del df_CCard_bal_numerical_agg,df_CCard_bal_cat_agg,df_CCard_balance,df_CCard_Bal_cat_group
gc.collect()

0

#### Installment Payments

In [94]:
df_Installment_payments = reduce_memory_usage(pd.read_csv('./data/installments_payments.csv', encoding= 'unicode_escape'))
print('Number of data points : ', df_Installment_payments.shape[0])
print('Number of features : ', df_Installment_payments.shape[1])
df_Installment_payments.head(1)

Memory usage of dataframe is 830.41 MB
Memory usage after optimization is: 311.40 MB
Decreased by 62.5%
Number of data points :  13605401
Number of features :  8


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.359863,6948.359863


In [95]:
df_cols = df_Installment_payments.loc[:, df_Installment_payments.columns.str.contains('INSTALMENT')]
for col in df_cols:
    print('column {} min value {} and max valaue {}'.format(col, df_cols[col].min(),df_cols[col].max()))

column NUM_INSTALMENT_VERSION min value 0.0 and max valaue 178.0
column NUM_INSTALMENT_NUMBER min value 1 and max valaue 277
column DAYS_INSTALMENT min value -2922.0 and max valaue -1.0
column AMT_INSTALMENT min value 0.0 and max valaue 3771487.75


In [96]:
df_Installment_payments['DAYS_INSTALMENT']=-df_Installment_payments['DAYS_INSTALMENT']

In [97]:
df_cols = df_Installment_payments.loc[:, df_Installment_payments.columns.str.contains('AMT')]
for col in df_cols:
    print('column {} min value {} and max valaue {}'.format(col, df_cols[col].min(),df_cols[col].max()))

column AMT_INSTALMENT min value 0.0 and max valaue 3771487.75
column AMT_PAYMENT min value 0.0 and max valaue 3771487.75


In [98]:
df_cols = df_Installment_payments.loc[:, df_Installment_payments.columns.str.contains('DAY')]
for col in df_cols:
    print('column {} min value {} and max valaue {}'.format(col, df_cols[col].min(),df_cols[col].max()))

column DAYS_INSTALMENT min value 1.0 and max valaue 2922.0
column DAYS_ENTRY_PAYMENT min value -4920.0 and max valaue -1.0


In [99]:
df_Installment_payments['DAYS_ENTRY_PAYMENT']=-df_Installment_payments['DAYS_ENTRY_PAYMENT']

In [100]:
#Check bureau and train have similar columns. if so, rename bureau columns, except ID columns
print('Similar columns~~>{}'.format(df_Installment_payments.columns.intersection(df_train.columns).tolist()))

Similar columns~~>['SK_ID_CURR']


In [101]:
num_columns=df_Installment_payments.select_dtypes(exclude=object).columns.to_list()
print('# of Numerical columns in previous application~~> {}'.format(len(num_columns)))

cat_columns=df_Installment_payments.select_dtypes(include=object).columns.to_list()
print('# of categorical columns in previous application~~> {}'.format(len(cat_columns)))

# of Numerical columns in previous application~~> 8
# of categorical columns in previous application~~> 0


In [102]:
# fill missing values
df_Installment_payments = fill_median_value(df_Installment_payments)

In [103]:
# Group by the applicant id, calculate aggregation statistics
df_Install_payment_numerical_agg = numeric_aggregate(df_Installment_payments.drop(columns = ['SK_ID_PREV']),
                                              group_var = 'SK_ID_CURR', df_name = 'Install_pay').reset_index(drop=True)
                                    
df_train=df_train.merge(df_Install_payment_numerical_agg,on='SK_ID_CURR', how='left' )

In [104]:
#check if any records with nan or inf value before 
#zero means no inf, nan values
df_train.replace([np.inf, -np.inf,np.NaN], 0, inplace=True)
#df_train[df_train.isin([np.nan, np.inf, -np.inf]).any(1)].shape[0]

In [105]:
gc.enable()
del df_Installment_payments,df_Install_payment_numerical_agg
gc.collect()

0

In [106]:
#df_train=df_train.reindex(index=df_train['SK_ID_CURR'])
df_train =  reduce_memory_usage(df_train)

Memory usage of dataframe is 2074.56 MB
Memory usage after optimization is: 668.64 MB
Decreased by 67.8%


In [107]:
df_train.shape

(307511, 896)

In [108]:
df_train.to_csv('./data/pre_processed_df_train.csv', sep='\t', encoding='utf-8')